In [3]:
# IMPORTACIONES
import sys
sys.path.append('..')
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import os

from utilities_amigocloud import AmigocloudFunctions
from utilities import generar_qr
from utilities import convertir_formato_fecha
from utilities import convertir_dict_obj

In [4]:
amigocloud = AmigocloudFunctions(API_AMIGOCLOUD_TOKEN_ADM)

In [5]:
# CONSTANTES
ID_PROYECTO = 34080
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

## generar reporte

In [ ]:
id_insp=126

In [ ]:
query = f'select ca.canhero, ca.maquina, ca.modelo, ca.codigo_maquina, ca.frente, insp.* from dataset_341171 insp\
        inner join dataset_341170 ca on ca.amigo_id=insp.maquina_ref_preven\
        where insp.id={id_insp}'
# ejecucion del query
select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')

In [6]:
def generar_reporte(id_insp):
    # consulta para extraer los datos de inspeccion por id concreto
    query = f'select ca.canhero, ca.maquina, ca.modelo, ca.codigo_maquina, ca.frente, insp.* from dataset_341171 insp\
        inner join dataset_341170 ca on ca.amigo_id=insp.maquina_ref_preven\
        where insp.id={id_insp}'
    # ejecucion del query
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    # verificar si se tiene valores para procesar
    if (len(select['data']) > 0):
        data = select['data'][0]
    else:
        print('Error, el registro de la evaluacion no tiene asigno maquina ni cañero asignado')
        return
    
    # modificacion de la fecha a formato dd/mm/aaaa
    data['fecha_registro'] = convertir_formato_fecha(data['fecha_registro'])

    # modificacion del nombre de las claves, con una k_ por delante, por que los campos booblenos (1_1, 2_2, etc) no se pueden trabajar bien
    data = {'k_'+key : value for (key, value) in data.items()}
    # convertir data(dict) en data
    insp = convertir_dict_obj(data, 'insp')

    if insp.k_codigo_maquina is None:
        print('Error, el registro de CAÑERO-MAQUINA no esta correcto, no tiene codigo maquina')
        return
    
    # buscar todas las fotos de un id especifico
    fotos_res = amigocloud.buscar_fotos(ID_PROYECTO,'gallery_57490', insp.k_amigo_id)
    # se selecciona solo la data de datos de todo lo retornado
    fotos = fotos_res['data']

    # ferifica si existe alguna foto sin s3_filename definido
    s3_fotos = [foto['s3_filename'] for foto in fotos]
    if None in s3_fotos:
        print('Error, existen fotos con s3_filename nulos')
        #return
    
    # crea el template para el reporte
    doc = DocxTemplate(os.path.join(RUTA_COMPLETA, 'templates/tpl_insp_cosechadoras-preventivo.docx'))

    # descargar lista de fotos, y convertir a inline del template doc
    lista_img_inline = amigocloud.descargar_convertir_fotos_inline(fotos, os.path.join(RUTA_COMPLETA, 'fotos'), doc)
    
    # generar texto contenido del qr
    texto_qr = f'''
    --UTEA--
    --PROGRAMA CAÑA LIMPIA--
    
    Id de Evaluacion: {insp.k_id}
    Fecha de Evaluacion: {insp.k_fecha_registro}
    Nombre de Canero: {insp.k_canhero.split(' / ')[1]}
    Codigo Canero: {insp.k_canhero.split(' / ')[0]}
    Maquina Cosechadora: {insp.k_maquina + '-' + insp.k_modelo}
    Codigo de Maquina: {insp.k_codigo_maquina}
    Calificacion de Evaluacion: {round(insp.k_nota_porcen*100, 2)}%'''

    # se genera el qr
    ruta_qr = generar_qr(texto_qr, os.path.join(RUTA_COMPLETA, '_imagenes'))
    # convertir el qr en inline
    qr_inline = amigocloud.convertir_img_to_inline(doc, ruta_qr, 30)

    # se crea el contexto de datos para pasar el template doc
    context = {'insp':insp, 'fotos':lista_img_inline, 'qr':qr_inline}
    # renderizar el doc
    doc.render(context)
    # crea lista con el codigo y nombre del canhero
    codigo_nombre = insp.k_canhero.split(' / ')
    # crea la ruta y nombre del informe a crear
    file_name = codigo_nombre[0] + '_EMPC_' + insp.k_fecha_registro.replace('/','-') + '_' + codigo_nombre[1] + '_' + str(insp.k_id)
    # se crea el documento
    doc.save(RUTA_COMPLETA + r'/informes/_' + file_name + '.docx')
    amigocloud.modificar_reporte_generado(ID_PROYECTO, id_insp)

In [7]:
def verificar_registros_nuevos():
    # ejecucion del query
    query = 'select id from dataset_341171 where informe_generado = false'
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    if select == None:
        print('EMPC: Error al listar todos los registros sin informe')
        return None
    # extrae la seccion de data
    data = select['data']
    if len(data) == 0:
        print('EMPC: No existen nuevos registros')
        return None
    #actualizar la nota de resgistros sin informe
    result = amigocloud.ejecutar_query_por_id(ID_PROYECTO, 6340)
    print(f'EMPC: Se ha actualizar la nota de {result} registros')
    # convierte data a lista de ids
    lista_ids = [dato['id'] for dato in data]
    for i in lista_ids:
        generar_reporte(i)

In [9]:
verificar_registros_nuevos()

EMPC: Se ha actualizar la nota de 21 registros
Lista de imagenes descargadas en blaco
Error, existen fotos con s3_filename nulos
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Lista de imagenes descargadas en blaco
Lista de imagenes descargadas en blaco
Error, existen fotos con s3_filename nulos
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Lista de imagenes descargadas en blaco
Lista de imagenes descargadas en blaco
Lista de imagenes descargadas en blaco
Error, existen fotos con s3_filename nulos
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Lista de imagenes descargadas en blaco
Error, existen fotos con s3_filename nulos
Foto sin 's3_filename'. Se omite.
Lista de imagenes descargadas en blaco
Error, existen fotos con s3_filename nulos
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filename'. Se omite.
Foto sin 's3_filen